In [1]:
import numpy as np
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_ALIGN_VERTICAL

In [2]:
# 定义函数
def docx_find_replace_text(doc, search_text, replace_text):
    for paragraph in doc.paragraphs:
        if search_text in paragraph.text:
            for run in paragraph.runs:
                if search_text in run.text:
                    run.text = run.text.replace(search_text, replace_text)

# docx_find_replace_text(doc, '«name»', 'new text')

# 替换编号
# def docx_find_replace_header(doc, search_text, replace_text):
#     for section in doc.sections:
#         header = section.header  # 获取节的页眉
#         for paragraph in header.paragraphs:  # 遍历页眉的每个段落
#             if search_text in paragraph.text:
#                 for run in paragraph.runs:
#                     if search_text in run.text:
#                         run.text = run.text.replace(search_text, replace_text)

# docx_find_replace_header(doc, '«c_id»', 'new text')

#确定样式
def set_cell_text_style(cell, pos): 
    cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER # 垂直居中
    for para in cell.paragraphs:
        if pos.lower() == "left":
            para.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
        elif pos.lower() == "mid":
            para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        elif pos.lower() == "right":
            para.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
        else:
            raise ValueError("Invalid position value. Please use 'left', 'mid' or 'right'.")
        for run in para.runs:
            run.font.size = Pt(10.5) # 五号字体
            run.font.name = u"宋体"
            run._element.rPr.rFonts.set(qn('w:eastAsia'), u'宋体')
    return cell

In [3]:
df = pd.read_excel('../应收预收-函证-汇总表.xlsx', skiprows=1, sheet_name='上海军陶应收函证')
df.dtypes

函证编号        object
主体          object
第三方名称       object
2020        object
2021        object
2022        object
期初余额.1     float64
借方发生额.1    float64
贷方发生额.1    float64
期末余额.1     float64
期初余额.2     float64
借方发生额.2    float64
贷方发生额.2    float64
期末余额.2     float64
期初余额.3     float64
借方发生额.3    float64
贷方发生额.3    float64
期末余额.3     float64
期初余额.4     float64
借方发生额.4    float64
贷方发生额.4    float64
期末余额.4     float64
函证地址        object
收件人         object
联系方式        object
dtype: object

In [4]:
comp_id = df['函证编号'].drop_duplicates().values
len(comp_id)

110

In [6]:
# 按照每一行去填就行
for c_id in comp_id:
    name = df[df['函证编号']==c_id]['第三方名称'].values[0]
    temp = df[df['函证编号']==c_id].iloc[:,[i for i in range(6,22)]].values.reshape(4,4)
    doc = Document('./example_应收预收.docx')  
    docx_find_replace_text(doc, '«name»', name)
    docx_find_replace_text(doc, '«c_id»', c_id)
    for i in range(4):
        row = doc.tables[0].rows[i+1]
        for j in range(4):
            row.cells[j+2].text = '{:,.2f}'.format(temp[i,j])
            set_cell_text_style(row.cells[j+2], 'right')
            
    doc.save(f'../4.1应收预付/{name}-应收预收函证.docx')
              


In [7]:
temp

array([[      0. ,       0. ,       0. ,       0. ],
       [      0. ,       0. ,       0. ,       0. ],
       [      0. ,   48002.4,       0. ,   48002.4],
       [  48002.4, 2169600. ,   48002.4, 2169600. ]])